# YOLO Model Training - champion icon detection

1. use `minimap_generator.py` generate minimap dataset
2. prepare YOLO style training and validation set
3. train model


In [1]:
import os
import sys
import shutil
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
import random
from tqdm import tqdm

sys.path.append('../')
from minimap_generator import ImageDrawer

print("Importing ImageDrawer")


Importing ImageDrawer


## 1. Setup


In [2]:
# motion blur + gaussian blur helper functions
import random
import cv2
import numpy as np

def motion_blur(img, k=3, angle=15):
    # build horizontal kernel and rotate
    kernel = np.zeros((k, k), dtype=np.float32)
    kernel[k // 2, :] = 1.0
    M = cv2.getRotationMatrix2D((k / 2 - 0.5, k / 2 - 0.5), angle, 1.0)
    kernel = cv2.warpAffine(kernel, M, (k, k))
    s = kernel.sum()
    if s > 0:
        kernel /= s
    return cv2.filter2D(img, -1, kernel)

def blur_icons(img_uint8, annotations, prob=0.3):
    """
    Apply motion blur + gaussian blur to the icon area of YOLO annotations with probability prob.
    img_uint8: HWC, uint8
    annotations: ['class cx cy w h'] list (normalized coordinates)
    """
    if random.random() >= prob:
        return img_uint8
    h, w = img_uint8.shape[:2]
    out = img_uint8.copy()
    for ann in annotations:
        parts = ann.strip().split()
        if len(parts) < 5:
            continue
        _, cx, cy, bw, bh = parts
        cx, cy, bw, bh = map(float, (cx, cy, bw, bh))
        x1 = int((cx - bw / 2) * w); y1 = int((cy - bh / 2) * h)
        x2 = int((cx + bw / 2) * w); y2 = int((cy + bh / 2) * h)
        x1 = max(0, x1); y1 = max(0, y1); x2 = min(w, x2); y2 = min(h, y2)
        if x2 <= x1 or y2 <= y1:
            continue
        patch = out[y1:y2, x1:x2]

        k_mb = 3
        angle = 15
        patch = motion_blur(patch, k=k_mb, angle=angle)

        k_g = 3
        if k_g % 2 == 0:
            k_g += 1
        patch = cv2.GaussianBlur(patch, (k_g, k_g), sigmaX=1)
        out[y1:y2, x1:x2] = patch
    return out

print("motion blur + gaussian blur helper functions loaded")


motion blur + gaussian blur helper functions loaded


In [5]:
# setup assets paths
champion_circle_icon_path = "../minimap_sources/champion_icons_circular_small"
minimap_path = "../minimap_sources/minimap_assets"
fog_path = "../minimap_sources/fog_assets"
misc_path = "../minimap_sources/misc_assets"

# output image size
resize = (640, 640)  # YOLO common size

# dataset parameters
num_train_samples = 5000  # training set sample number
num_val_samples = 1000     # validation set sample number
output_base_dir = "./yolo_dataset"  # dataset root directory

print("paths setup completed")
print(f"training set sample number: {num_train_samples}")
print(f"validation set sample number: {num_val_samples}")
print(f"output image size: {resize}")


paths setup completed
training set sample number: 5000
validation set sample number: 1000
output image size: (640, 640)


## 2. Initialize ImageDrawer


In [6]:
try:
    drawer = ImageDrawer(
        champion_circle_icon_path=champion_circle_icon_path,
        minimap_path=minimap_path,
        fog_path=fog_path,
        misc_path=misc_path,
        resize=resize
    )
    print("ImageDrawer initialized successfully")
    print(f"loaded {len(drawer.champion_icons)} champion icons")
    print(f"loaded {len(drawer.minimaps)} minimap backgrounds")
    print(f"loaded {len(drawer.fogs)} fog images")
    print(f"loaded {len(drawer.miscs)} other assets")
except Exception as e:
    print(f"initialization failed: {e}")
    import traceback
    traceback.print_exc()


ImageDrawer initialized successfully
loaded 172 champion icons
loaded 18 minimap backgrounds
loaded 10 fog images
loaded 263 other assets


## 3. Generate Dataset


In [7]:
train_images_dir = os.path.join(output_base_dir, "train", "images")
train_labels_dir = os.path.join(output_base_dir, "train", "labels")
val_images_dir = os.path.join(output_base_dir, "val", "images")
val_labels_dir = os.path.join(output_base_dir, "val", "labels")

os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

print("dataset directory structure created")


dataset directory structure created


In [ ]:
# generate training set (with 30% probability of icon motion blur + gaussian blur)
print("generating training set...")
for i in tqdm(range(num_train_samples), desc="generating training set"):
    try:
        final_image, yolo_data = drawer.generate_data(i, allow_overlap=False)
        
        # save image
        img_np = final_image.numpy()
        if img_np.shape[0] == 3:  # CHW format
            img_np = img_np.transpose(1, 2, 0)  # convert to HWC format
        img_np = np.clip(img_np, 0, 1)
        img_np = (img_np * 255).astype(np.uint8)

        # blur icons with 30% probability
        img_np = blur_icons(img_np, yolo_data['annotations'], prob=0.3)

        img_pil = Image.fromarray(img_np)
        
        image_filename = f"minimap_{i:06d}.jpg"
        image_path = os.path.join(train_images_dir, image_filename)
        img_pil.save(image_path)
        
        # save YOLO format annotation file
        annotation_filename = f"minimap_{i:06d}.txt"
        annotation_path = os.path.join(train_labels_dir, annotation_filename)
        with open(annotation_path, 'w') as f:
            for ann_str in yolo_data['annotations']:
                f.write(ann_str + '\n')
    except Exception as e:
        print(f"generating training sample {i} failed: {e}")

print(f"training set generated successfully, {num_train_samples} samples")


开始生成训练集（含模糊增强）...


生成训练集: 100%|██████████| 5000/5000 [03:16<00:00, 25.47it/s]

训练集生成完成！共 5000 个样本


In [ ]:
# 生成验证集
print("generating validation set...")
for i in tqdm(range(num_val_samples), desc="generating validation set"):
    try:
        # avoid duplicate with training set
        index = num_train_samples + i
        final_image, yolo_data = drawer.generate_data(index, allow_overlap=False)
        
        # save image
        img_np = final_image.numpy()
        if img_np.shape[0] == 3:  # CHW format
            img_np = img_np.transpose(1, 2, 0)  # convert to HWC format
        img_np = np.clip(img_np, 0, 1)
        img_np = (img_np * 255).astype(np.uint8)
        img_pil = Image.fromarray(img_np)
        
        image_filename = f"minimap_{i:06d}.jpg"
        image_path = os.path.join(val_images_dir, image_filename)
        img_pil.save(image_path)
        
        # save YOLO format annotation file
        annotation_filename = f"minimap_{i:06d}.txt"
        annotation_path = os.path.join(val_labels_dir, annotation_filename)
        with open(annotation_path, 'w') as f:
            for ann_str in yolo_data['annotations']:
                f.write(ann_str + '\n')
    except Exception as e:
        print(f"generating validation sample {i} failed: {e}")

print(f"validation set generated successfully, {num_val_samples} samples")


开始生成验证集...


生成验证集: 100%|██████████| 1000/1000 [00:39<00:00, 25.26it/s]

验证集生成完成！共 1000 个样本


## 4. Create YOLO Dataset Config


In [8]:
# create classes.txt file
classes_file = os.path.join(output_base_dir, "classes.txt")
with open(classes_file, 'w', encoding='utf-8') as f:
    max_id = max(drawer.id_to_champion.keys())
    for class_id in range(max_id):
        champion_id = class_id + 1
        if champion_id in drawer.id_to_champion:
            champion_name = drawer.id_to_champion[champion_id]
            f.write(f"{champion_name}\n")
        else:
            f.write(f"unknown_{champion_id}\n")

print(f"classes.txt file created: {classes_file}")
print(f"total {max_id} classes")

# show first 10 classes
print("\nfirst 10 classes:")
with open(classes_file, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f.readlines()[:10]):
        print(f"  Class {i}: {line.strip()}")


classes.txt file created: ./yolo_dataset\classes.txt
total 172 classes

first 10 classes:
  Class 0: Aatrox
  Class 1: Ahri
  Class 2: Akali
  Class 3: Akshan
  Class 4: Alistar
  Class 5: Ambessa
  Class 6: Amumu
  Class 7: Anivia
  Class 8: Annie
  Class 9: Aphelios


In [9]:
# create data.yaml file
data_yaml_path = os.path.join(output_base_dir, "data.yaml")

# get absolute path
abs_base_dir = os.path.abspath(output_base_dir)
abs_train_images = os.path.abspath(train_images_dir)
abs_val_images = os.path.abspath(val_images_dir)
abs_classes = os.path.abspath(classes_file)

# read number of classes
num_classes = max(drawer.id_to_champion.keys())

yaml_content = f"""# YOLO Dataset Configuration
# Path to dataset root
path: {abs_base_dir}

# Train and val images
train: {abs_train_images}
val: {abs_val_images}

# Number of classes
nc: {num_classes}

# Class names
names:
"""
# read class names
with open(classes_file, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f.readlines()):
        class_name = line.strip()
        yaml_content += f"  {i}: {class_name}\n"

with open(data_yaml_path, 'w', encoding='utf-8') as f:
    f.write(yaml_content)

print(f"YOLO dataset config file created: {data_yaml_path}")
print(f"number of classes: {num_classes}")


YOLO dataset config file created: ./yolo_dataset\data.yaml
number of classes: 172


## 5. Import YOLO

Using YOLOv8 from ultralytics


In [10]:
# !pip install ultralytics

# import YOLO
try:
    from ultralytics import YOLO
    print("YOLO library imported successfully!")
except ImportError:
    print("Please install ultralytics: pip install ultralytics")
    print("or run: !pip install ultralytics")


YOLO library imported successfully!


## 6. Train YOLO Model


In [11]:
model_size = 's'
epochs = 150
batch_size = 64
imgsz = 640
device = 'cuda'
workers = 4
amp = True
cache = 'disk'
close_mosaic = 20

print(f"training parameters:")
print(f"  model size: YOLOv8{model_size}")
print(f"  epochs: {epochs}")
print(f"  batch size: {batch_size}")
print(f"  image size: {imgsz}")
print(f"  device: {device}")
print(f"  workers: {workers}")
print(f"  mixed precision training: {amp}")
print(f"  cache: {cache}")
print(f"  close mosaic: {close_mosaic}")


training parameters:
  model size: YOLOv8s
  epochs: 150
  batch size: 64
  image size: 640
  device: cuda
  workers: 4
  mixed precision training: True
  cache: disk
  close mosaic: 20


In [ ]:
model = YOLO(f'yolov8{model_size}.pt') 


results = model.train(
    data=data_yaml_path,      # dataset config file path
    epochs=epochs,            # number of epochs
    imgsz=imgsz,              # image size
    batch=batch_size,         # batch size
    device=device,            # device
    project='yolo_runs',      # project name (cannot contain path separators)
    name='minimap_detection', # experiment name
    exist_ok=True,            # if directory exists, overwrite
    save=True,                # save checkpoints
    plots=True,               # generate training plots
    workers=workers,
    amp=amp,
    cache=cache,
    close_mosaic=close_mosaic
)

print("Training completed!")


New https://pypi.org/project/ultralytics/8.3.235 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.0  Python-3.11.13 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32607MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=./yolo_dataset\data.yaml, epochs=150, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=disk, device=cuda, workers=4, project=yolo_runs, name=minimap_detection, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=20, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=Non

wandb: Currently logged in as: slsteven (slsteven-Boston University) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed 


train: Scanning F:\____Project\CV_project\yolo_3rd_try\yolo_dataset\train\labels... 5000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:04<00:00, 1159.72it/s]


train: New cache created: F:\____Project\CV_project\yolo_3rd_try\yolo_dataset\train\labels.cache


...


150 epochs completed in 1.209 hours.
Optimizer stripped from yolo_runs\minimap_detection\weights\last.pt, 20.1MB
Optimizer stripped from yolo_runs\minimap_detection\weights\best.pt, 20.1MB

Validating yolo_runs\minimap_detection\weights\best.pt...
Ultralytics 8.3.0  Python-3.11.13 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32607MiB)
Model summary (fused): 186 layers, 9,894,228 parameters, 0 gradients, 23.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:07<00:00,  1.10it/s]


                   all       1000      10000      0.999      0.995      0.995      0.995
                Aatrox         65         65      0.999          1      0.995      0.995
                  Ahri         49         49      0.994          1      0.995      0.995
                 Akali         51         51      0.997          1      0.995      0.995
                Akshan         70         70      0.996      0.986      0.995      0.994
               Alistar         64         64          1      0.992      0.995      0.995
               Ambessa         61         61          1      0.985      0.995      0.995
                 Amumu         47         47      0.998          1      0.995      0.995
                Anivia         62         62      0.998          1      0.995      0.995
                 Annie         57         57      0.999          1      0.995      0.995
              Aphelios         62         62      0.999          1      0.995      0.995
                  Ash

lr/pg0,███▇▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▁▁▁
lr/pg1,██████▇▇▇▇▇▇▇▇▇▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▁
lr/pg2,████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁
metrics/mAP50(B),▁▇██████████████████████████████████████
metrics/mAP50-95(B),▁▅██████████████████████████████████████
metrics/precision(B),▁▄▃▅▆▆▇▇█▇█▇████████████████████████████
metrics/recall(B),▁███████████████████████████████████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
+6,...


训练完成！


## 7. Evaluate Model Performance


In [12]:
# load best model weights
best_model_path = './yolo_runs/minimap_detection/weights/best.pt'
model = YOLO(best_model_path)

# evaluate on validation set
metrics = model.val(data=data_yaml_path)
print("\nValidation set evaluation results:")
print(f"  mAP50: {metrics.box.map50:.4f}")
print(f"  mAP50-95: {metrics.box.map:.4f}")
print(f"  precision: {metrics.box.mp:.4f}")
print(f"  recall: {metrics.box.mr:.4f}")


Ultralytics 8.3.0  Python-3.11.13 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32607MiB)
Model summary (fused): 186 layers, 9,894,228 parameters, 0 gradients, 23.7 GFLOPs


val: Scanning D:\STUDY\2025 Fall\Computer Vision\2025Fall_ComputerVision\minimap\yolo\yolo_dataset\val\labels... 1000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:00<00:00, 1365.27it/s]

val: New cache created: D:\STUDY\2025 Fall\Computer Vision\2025Fall_ComputerVision\minimap\yolo\yolo_dataset\val\labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:06<00:00,  9.82it/s]


                   all       1000      10000      0.999      0.995      0.995      0.995
                Aatrox         65         65      0.999          1      0.995      0.995
                  Ahri         49         49      0.994          1      0.995      0.995
                 Akali         51         51      0.997          1      0.995      0.995
                Akshan         70         70      0.996      0.986      0.995      0.994
               Alistar         64         64          1      0.991      0.995      0.995
               Ambessa         61         61          1      0.985      0.995      0.995
                 Amumu         47         47      0.998          1      0.995      0.995
                Anivia         62         62      0.998          1      0.995      0.995
                 Annie         57         57      0.999          1      0.995      0.995
              Aphelios         62         62      0.999          1      0.995      0.995
                  Ash

## 8. Test Model Prediction


In [15]:
# generate a test image
test_index = num_train_samples + num_val_samples + 1
test_image, test_yolo_data = drawer.generate_data(test_index, allow_overlap=False)

# save test image
test_img_np = test_image.numpy()
if test_img_np.shape[0] == 3:
    test_img_np = test_img_np.transpose(1, 2, 0)
test_img_np = np.clip(test_img_np, 0, 1)
test_img_np = (test_img_np * 255).astype(np.uint8)
test_img_pil = Image.fromarray(test_img_np)

test_image_path = './test_minimap.jpg'
test_img_pil.save(test_image_path)

print(f"Test image saved: {test_image_path}")
print(f"Number of true annotations: {len(test_yolo_data['annotations'])}")


Test image saved: ./test_minimap.jpg
Number of true annotations: 10


In [16]:
results = model.predict(
    source=test_image_path,
    conf=0.25,  # confidence threshold
    iou=0.45,   # IoU threshold
    save=True,
    show_labels=True,
    show_conf=True,
)

# display prediction results
print(f"\nNumber of detected champions: {len(results[0].boxes)}")
if len(results[0].boxes) > 0:
    print("\nDetection results:")
    for i, box in enumerate(results[0].boxes):
        class_id = int(box.cls[0])
        confidence = float(box.conf[0])
        champion_id = class_id + 1
        if champion_id in drawer.id_to_champion:
            champion_name = drawer.id_to_champion[champion_id]
            print(f"  {i+1}. {champion_name} (confidence: {confidence:.3f})")
        else:
            print(f"  {i+1}. Class {class_id} (confidence: {confidence:.3f})")



image 1/1 d:\STUDY\2025 Fall\Computer Vision\2025Fall_ComputerVision\minimap\yolo\test_minimap.jpg: 640x640 1 Aphelios, 1 Blitzcrank, 1 Brand, 1 Kassadin, 1 KogMaw, 1 Malzahar, 1 Poppy, 1 Udyr, 1 Vi, 1 Zed, 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict

Number of detected champions: 10

Detection results:
  1. Blitzcrank (confidence: 0.984)
  2. Aphelios (confidence: 0.983)
  3. Vi (confidence: 0.982)
  4. Malzahar (confidence: 0.982)
  5. Udyr (confidence: 0.979)
  6. KogMaw (confidence: 0.979)
  7. Poppy (confidence: 0.978)
  8. Kassadin (confidence: 0.977)
  9. Zed (confidence: 0.976)
  10. Brand (confidence: 0.971)


## 9. Save Model


In [ ]:
best_model_path = './yolo_runs/minimap_detection/weights/best.pt'
last_model_path = './yolo_runs/minimap_detection/weights/last.pt'

if os.path.exists(best_model_path):
    file_size = os.path.getsize(best_model_path) / (1024 * 1024)  # MB
    print(f"Best model saved: {best_model_path}")
    print(f"Model size: {file_size:.2f} MB")
else:
    print("Best model file not found")

if os.path.exists(last_model_path):
    file_size = os.path.getsize(last_model_path) / (1024 * 1024)  # MB
    print(f"Last model saved: {last_model_path}")
    print(f"Model size: {file_size:.2f} MB")
else:
    print("Last model file not found")


最佳模型已保存: ./yolo_runs/minimap_detection/weights/best.pt
模型大小: 19.17 MB
最后模型已保存: ./yolo_runs/minimap_detection/weights/last.pt
模型大小: 19.17 MB
